In [1]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from src.consts import *

load_dotenv(override=True)
openai = OpenAI()
anthropic = Anthropic() 

# Load the data files
oracle_df = pd.read_csv('ThePauperCube_oracle_with_pt.csv')
print(f"Loaded {len(oracle_df)} cards from oracle_df")
print(f"Columns available: {list(oracle_df.columns)}")
oracle_df.head()

Loaded 450 cards from oracle_df
Columns available: ['name', 'CMC', 'Type', 'Color', 'Color Category', 'Oracle Text', 'tags', 'MTGO ID', 'Power', 'Toughness']


,name,CMC,Type,Color,Color Category,Oracle Text,tags,MTGO ID,Power,Toughness
0,Boros Elite,1,Creature - Human Soldier,W,White,Battalion — Whenever this creature and at leas...,NaN,120547.0,1.0,1.0
1,Deftblade Elite,1,Creature - Human Soldier,W,White,"Provoke (Whenever this creature attacks, you m...",NaN,18617.0,1.0,1.0
2,Doomed Traveler,1,Creature - Human Soldier,W,White,"When this creature dies, create a 1/1 white Sp...",GTC Update;token generator;WR tokens;WG tokens,42650.0,1.0,1.0
3,Elite Vanguard,1,Creature - Human Soldier,W,White,NaN,EMA Update,60565.0,2.0,1.0
4,Faerie Guidemother,1,Creature - Faerie,W,White,Flying // Target creature gets +2/+1 and gains...,ELD Update,78110.0,1.0,1.0


# Theme Validation
Let's check if we have enough cards available for each theme in our jumpstart cube.

# Deck Construction
Now let's test the deck construction function to build actual jumpstart decks from our themes.

In [2]:
# Test the refactored deck construction function

# Build all jumpstart decks using the new refactored version
from src.construct import construct_jumpstart_decks, print_detailed_deck_analysis, CardConstraints, analyze_deck_composition

# Create constraints with custom target deck size
constraints = CardConstraints(target_deck_size=13)

print("🚀 Starting deck construction with refactored algorithm...")
deck_dataframes = construct_jumpstart_decks(oracle_df, constraints=constraints)

# Generate analysis first, then print detailed analysis
analysis = analyze_deck_composition(deck_dataframes)
print_detailed_deck_analysis(deck_dataframes, analysis)

🚀 Starting deck construction with refactored algorithm...
🏗️ CONSTRUCTING JUMPSTART DECKS

🔒 Phase 0: Core card reservation
Ensuring each theme gets its defining cards before general competition...

🎯 White Soldiers: Reserving core cards
  ⚠️  No core cards found meeting criteria (score ≥6.0)

🎯 White Equipment: Reserving core cards
  ✅ Mandibular Kite           |  12.8 pts | Creature Artifact - ...
  ✅ Glimmerlight              |  11.8 pts | Artifact - Equipment
  ✅ Vulshok Morningstar       |  11.8 pts | Artifact - Equipment
  ✅ Ancestral Blade           |  11.4 pts | Creature Artifact - ...
  ✅ Flayer Husk               |  11.4 pts | Creature Artifact - ...
  📦 Reserved 5 core cards

🎯 White Angels: Reserving core cards
  ⚠️  No core cards found meeting criteria (score ≥6.0)

🎯 White Weenies: Reserving core cards
  ✅ Boros Elite               |   7.8 pts | Creature - Human Sol...
  ✅ Miner's Guidewing         |   7.8 pts | Creature - Bird
  ✅ Cartouche of Solidarity   |   7.8 pts | 

In [3]:
from src.export import export_cube_to_csv


export_cube_to_csv(deck_dataframes, 'jumpstart_decks.csv')

Exporting jumpstart cube to jumpstart_decks.csv...
✅ Successfully exported 390 cards to jumpstart_decks.csv

📊 Export Summary:
Total cards: 390
Number of decks: 30

Deck breakdown:
  White Soldiers: 13 cards
  White Equipment: 13 cards
  Boros Equipment Aggro: 13 cards
  Golgari Graveyard Value: 13 cards
  Izzet Spells Matter: 13 cards
  Orzhov Lifedrain: 13 cards
  Selesnya Tokens: 13 cards
  Gruul Big Creatures: 13 cards
  Rakdos Aggro: 13 cards
  Dimir Mill: 13 cards
  ... and 20 more decks


'jumpstart_decks.csv'

In [4]:
# Import validation functions and run card uniqueness validation
from src.validation import validate_card_uniqueness, validate_deck_constraints, validate_jumpstart_cube, display_validation_summary

# Run the validation
validation_result = validate_card_uniqueness(deck_dataframes)

🔍 VALIDATING CARD UNIQUENESS
📊 VALIDATION RESULTS:
Total cards across all decks: 390
Unique cards used: 390
Duplicate cards found: 0

✅ VALIDATION PASSED!
All 390 cards are used exactly once.


In [5]:
# Additional analysis using the imported validation functions
from src.validation import analyze_card_distribution

# Run the distribution analysis
distribution_analysis = analyze_card_distribution(deck_dataframes, oracle_df)


📈 CARD DISTRIBUTION ANALYSIS
📊 OVERALL STATISTICS:
Total cards available: 450
Total cards used: 390
Cards unused: 60
Usage rate: 86.7%

🎨 USAGE BY COLOR:
  White    :  61/ 67 cards ( 91.0%)
  Blue     :  61/ 66 cards ( 92.4%)
  Black    :  55/ 66 cards ( 83.3%)
  Red      :  60/ 68 cards ( 88.2%)
  Green    :  65/ 66 cards ( 98.5%)
  Colorless:  32/ 54 cards ( 59.3%)

🎯 DECK COMPLETENESS:
Complete decks (13 cards): 30
Incomplete decks: 0

📋 UNUSED CARDS ANALYSIS:
Unused creatures: 23
Unused lands: 21
Unused spells: 16

Sample unused cards:
  • Search Party Captain (Creature - Human Soldier) - W
  • Alabaster Host Intercessor (Creature - Phyrexian Samurai) - W
  • Borrowed Grace (Instant) - W
  • Prismatic Strands (Instant) - W
  • Imperial Oath (Creature Sorcery) - W
  • Oblivion Ring (Enchantment) - W
  • Aethersnipe (Creature - Elemental) - U
  • Bubble Snare (Enchantment - Aura) - U
  • Unable to Scream (Enchantment - Aura) - U
  • Weakstone's Subjugation (Enchantment - Aura) - U


# Generated
 
Below is AI

In [6]:
# Analyze Azorius Control Theme Composition
print("⚖️ AZORIUS CONTROL THEME ANALYSIS")
print("=" * 50)

# Get the Azorius Control deck if it exists
theme_name = "Azorius Control"
if theme_name in deck_dataframes:
    azorius_control_deck = deck_dataframes[theme_name]
    
    print(f"📊 DECK COMPOSITION:")
    print(f"   • Total cards: {len(azorius_control_deck)}")
    
    # Determine the correct type column
    type_column = None
    for col in ['Type', 'type_line', 'type', 'types', 'card_type']:
        if col in azorius_control_deck.columns:
            type_column = col
            break
    
    if type_column:
        print(f"\n👥 CREATURE TYPE ANALYSIS:")
        creature_cards = azorius_control_deck[azorius_control_deck[type_column].str.contains('Creature', na=False)]
        total_creatures = len(creature_cards)
        
        print(f"   • Total creatures: {total_creatures}")
        
        # Show all creatures
        if len(creature_cards) > 0:
            print(f"\n👤 CREATURES:")
            for _, card in creature_cards.iterrows():
                mana_cost = card.get('CMC', 'N/A')
                power = card.get('Power', 'N/A')
                toughness = card.get('Toughness', 'N/A')
                oracle_text = str(card.get('Oracle Text', ''))[:80]
                print(f"   • {card['name']} (CMC {mana_cost}, {power}/{toughness}) - {card[type_column]}")
                print(f"     Text: {oracle_text}{'...' if len(str(card.get('Oracle Text', ''))) > 80 else ''}")
                print()
        
        # Analyze non-creature spells
        non_creature_cards = azorius_control_deck[~azorius_control_deck[type_column].str.contains('Creature', na=False)]
        if len(non_creature_cards) > 0:
            print(f"\n📜 NON-CREATURE SPELLS:")
            for _, card in non_creature_cards.iterrows():
                mana_cost = card.get('CMC', 'N/A')
                oracle_text = str(card.get('Oracle Text', ''))[:80]
                print(f"   • {card['name']} (CMC {mana_cost}) - {card[type_column]}")
                print(f"     Text: {oracle_text}{'...' if len(str(card.get('Oracle Text', ''))) > 80 else ''}")
                print()
        
        # Analyze color distribution
        print(f"\n🎨 COLOR ANALYSIS:")
        if 'Color' in azorius_control_deck.columns:
            color_counts = azorius_control_deck['Color'].value_counts()
            for color, count in color_counts.items():
                percentage = (count / len(azorius_control_deck)) * 100
                print(f"   • {color}: {count} cards ({percentage:.1f}%)")
        
        # Check for control synergies and effects
        print(f"\n🔍 CONTROL SYNERGY ANALYSIS:")
        counterspell_count = 0
        removal_count = 0
        card_draw_count = 0
        board_wipe_count = 0
        win_condition_count = 0
        
        control_synergy_cards = []
        
        for _, card in azorius_control_deck.iterrows():
            oracle_text = str(card.get('Oracle Text', '')).lower()
            card_name = str(card.get('name', '')).lower()
            
            synergy_reasons = []
            card_synergies = 0
            
            # Check for counterspells
            if any(keyword in oracle_text for keyword in ['counter target', 'counter spell', 'counter']):
                if any(spell_type in oracle_text for spell_type in ['spell', 'target']):
                    counterspell_count += 1
                    synergy_reasons.append("counterspell")
                    card_synergies += 1
            
            # Check for removal spells
            if any(keyword in oracle_text for keyword in ['destroy', 'exile', 'return', 'bounce']):
                if any(target in oracle_text for target in ['target', 'creature', 'permanent']):
                    removal_count += 1
                    synergy_reasons.append("removal")
                    card_synergies += 1
            
            # Check for card draw
            if any(keyword in oracle_text for keyword in ['draw', 'scry', 'look']):
                if any(draw_word in oracle_text for draw_word in ['card', 'cards', 'library']):
                    card_draw_count += 1
                    synergy_reasons.append("card draw")
                    card_synergies += 1
            
            # Check for board wipes
            if any(keyword in oracle_text for keyword in ['all creatures', 'each creature', 'destroy all']):
                board_wipe_count += 1
                synergy_reasons.append("board wipe")
                card_synergies += 1
            
            # Check for win conditions (big creatures, planeswalkers, etc.)
            cmc = card.get('CMC', 0)
            if 'creature' in str(card.get(type_column, '')).lower():
                if cmc >= 4 or any(keyword in oracle_text for keyword in ['flying', 'vigilance', 'lifelink']):
                    win_condition_count += 1
                    synergy_reasons.append("win condition")
                    card_synergies += 1
            elif 'planeswalker' in str(card.get(type_column, '')).lower():
                win_condition_count += 1
                synergy_reasons.append("win condition")
                card_synergies += 1
            
            if card_synergies > 0:
                control_synergy_cards.append({
                    'name': card['name'],
                    'reasons': synergy_reasons
                })
        
        print(f"   • Cards with counterspells: {counterspell_count}")
        print(f"   • Cards with removal: {removal_count}")
        print(f"   • Cards with card draw: {card_draw_count}")
        print(f"   • Cards with board wipes: {board_wipe_count}")
        print(f"   • Win condition cards: {win_condition_count}")
        
        if control_synergy_cards:
            print(f"\n⚖️ CONTROL SYNERGY BREAKDOWN:")
            for card_info in control_synergy_cards:
                print(f"   • {card_info['name']}: {', '.join(card_info['reasons'])}")
        
        # Overall theme assessment
        print(f"\n⚖️  THEME MATCHING ASSESSMENT:")
        total_score = 0
        max_score = 4
        
        # Control tools density (25% of score)
        control_tools = counterspell_count + removal_count + board_wipe_count
        if control_tools >= 6:
            print("   ✅ Control Tools: EXCELLENT (6+ counters/removal/wipes)")
            total_score += 1
        elif control_tools >= 4:
            print("   ⚠️  Control Tools: GOOD (4-5 control spells)")
            total_score += 0.75
        elif control_tools >= 2:
            print("   ⚠️  Control Tools: FAIR (2-3 control spells)")
            total_score += 0.5
        else:
            print("   ❌ Control Tools: POOR (few control spells)")
        
        # Color consistency (25% of score)
        wu_cards = 0
        if 'Color' in azorius_control_deck.columns:
            wu_cards = len(azorius_control_deck[
                (azorius_control_deck['Color'].str.contains('W', na=False)) |
                (azorius_control_deck['Color'].str.contains('U', na=False)) |
                (azorius_control_deck['Color'].str.contains('WU', na=False))
            ])
        wu_percentage = (wu_cards / len(azorius_control_deck)) * 100
        if wu_percentage >= 80:
            print("   ✅ Color Identity: EXCELLENT (80%+ white/blue)")
            total_score += 1
        elif wu_percentage >= 60:
            print("   ⚠️  Color Identity: GOOD (60%+ white/blue)")
            total_score += 0.75
        else:
            print("   ❌ Color Identity: NEEDS IMPROVEMENT (<60% white/blue)")
        
        # Card advantage (25% of score)
        if card_draw_count >= 3:
            print("   ✅ Card Advantage: EXCELLENT (3+ draw effects)")
            total_score += 1
        elif card_draw_count >= 2:
            print("   ⚠️  Card Advantage: GOOD (2 draw effects)")
            total_score += 0.75
        elif card_draw_count >= 1:
            print("   ⚠️  Card Advantage: FAIR (1 draw effect)")
            total_score += 0.5
        else:
            print("   ❌ Card Advantage: POOR (no draw effects)")
        
        # Control strategy fit (25% of score)
        creature_ratio = total_creatures / len(azorius_control_deck) * 100
        if creature_ratio <= 30 and win_condition_count >= 2:
            print("   ✅ Control Strategy: EXCELLENT (few creatures, good win cons)")
            total_score += 1
        elif creature_ratio <= 40 and control_tools >= 3:
            print("   ⚠️  Control Strategy: GOOD (decent control density)")
            total_score += 0.75
        elif creature_ratio <= 50:
            print("   ⚠️  Control Strategy: FAIR (moderate creature count)")
            total_score += 0.5
        else:
            print("   ❌ Control Strategy: POOR (too creature-heavy for control)")
        
        final_percentage = (total_score / max_score) * 100
        print(f"\n🏆 OVERALL THEME SCORE: {final_percentage:.1f}% ({total_score:.2f}/{max_score})")
        
        if final_percentage >= 85:
            print("   🌟 EXCELLENT theme match!")
        elif final_percentage >= 70:
            print("   ✅ GOOD theme match")
        elif final_percentage >= 50:
            print("   ⚠️  FAIR theme match - could use improvement")
        else:
            print("   ❌ POOR theme match - needs significant work")
            
    else:
        print("   ❌ Could not find type column in the data")
        
else:
    print(f"❌ Theme '{theme_name}' not found in deck_dataframes")
    print(f"Available themes: {', '.join(sorted(deck_dataframes.keys()))}")

⚖️ AZORIUS CONTROL THEME ANALYSIS
📊 DECK COMPOSITION:
   • Total cards: 13

👥 CREATURE TYPE ANALYSIS:
   • Total creatures: 2

👤 CREATURES:
   • Judge's Familiar (CMC 1, 1.0/1.0) - Creature - Bird
     Text: Flying | Sacrifice this creature: Counter target instant or sorcery spell unless...

   • Filigree Familiar (CMC 3, 2.0/2.0) - Artifact Creature - Fox
     Text: When this creature enters, you gain 2 life. | When this creature dies, draw a ca...


📜 NON-CREATURE SPELLS:
   • Skybridge Towers (CMC 0) - Land
     Text: This land enters tapped. | {T}: Add {W} or {U}. | {2}{W}{U}, {T}, Sacrifice this...

   • Azorius Chancery (CMC 0) - Land
     Text: This land enters tapped. | When this land enters, return a land you control to i...

   • Tranquil Cove (CMC 0) - Land
     Text: This land enters tapped. | When this land enters, you gain 1 life. | {T}: Add {W...

   • Momentary Blink (CMC 2) - Instant
     Text: Exile target creature you control, then return it to the battlefield under 

In [7]:
# Analyze Unassigned White/Blue Cards for Better Azorius Control Options
print("🔍 UNASSIGNED AZORIUS CARDS ANALYSIS")
print("=" * 60)

# Get all assigned cards across all decks
assigned_cards = set()
for theme, deck_df in deck_dataframes.items():
    for card_name in deck_df['name']:
        assigned_cards.add(card_name)

# Filter for unassigned white/blue cards (including multicolor WU)
unassigned_wu_cards = oracle_df[
    (~oracle_df['name'].isin(assigned_cards)) & 
    (
        (oracle_df['Color'].str.contains('W', na=False)) |
        (oracle_df['Color'].str.contains('U', na=False)) |
        (oracle_df['Color'].str.contains('WU', na=False)) |
        (oracle_df['Color'].str.contains('UW', na=False))
    )
].copy()

print(f"📊 UNASSIGNED WHITE/BLUE CARDS: {len(unassigned_wu_cards)} total")

# Determine the correct type column
type_column = None
for col in ['Type', 'type_line', 'type', 'types', 'card_type']:
    if col in unassigned_wu_cards.columns:
        type_column = col
        break

if type_column and len(unassigned_wu_cards) > 0:
    # Analyze card types for control potential
    creature_cards = unassigned_wu_cards[unassigned_wu_cards[type_column].str.contains('Creature', na=False)]
    instant_cards = unassigned_wu_cards[unassigned_wu_cards[type_column].str.contains('Instant', na=False)]
    sorcery_cards = unassigned_wu_cards[unassigned_wu_cards[type_column].str.contains('Sorcery', na=False)]
    
    print(f"\n🎯 CARD TYPE BREAKDOWN:")
    print(f"   • Unassigned creatures: {len(creature_cards)}")
    print(f"   • Unassigned instants: {len(instant_cards)}")
    print(f"   • Unassigned sorceries: {len(sorcery_cards)}")
    
    # Analyze potential control themes based on oracle text
    print(f"\n🎯 CONTROL THEME ANALYSIS OF UNASSIGNED CARDS:")
    
    control_keywords = {
        'Counterspells': ['counter target', 'counter spell', 'counter', 'negate', 'cancel'],
        'Removal/Bounce': ['destroy', 'exile', 'return to hand', 'bounce', 'unsummon', 'path to exile'],
        'Board Wipes': ['destroy all', 'all creatures', 'each creature', 'wrath', 'board wipe'],
        'Card Draw/Advantage': ['draw', 'scry', 'look', 'search', 'tutor', 'library'],
        'Win Conditions': ['flying', 'vigilance', 'lifelink', 'planeswalker', 'serra', 'angel'],
        'Control Utility': ['tap', 'untap', 'prevent', 'protection', 'hexproof', 'indestructible'],
        'Lifegain/Stabilization': ['gain life', 'you gain', 'prevent damage', 'fog']
    }
    
    theme_scores = {}
    theme_cards = {}
    
    for theme_name, keywords in control_keywords.items():
        matching_cards = []
        score = 0
        
        for _, card in unassigned_wu_cards.iterrows():
            oracle_text = str(card.get('Oracle Text', '')).lower()
            card_name = str(card.get('name', '')).lower()
            card_type = str(card.get(type_column, '')).lower()
            
            matches = []
            for keyword in keywords:
                if keyword in oracle_text or keyword in card_name or keyword in card_type:
                    matches.append(keyword)
            
            if matches:
                matching_cards.append({
                    'name': card['name'],
                    'cmc': card.get('CMC', 'N/A'),
                    'type': card.get(type_column, ''),
                    'color': card.get('Color', ''),
                    'matches': matches,
                    'text': oracle_text[:80] + ('...' if len(oracle_text) > 80 else '')
                })
                score += len(matches)
        
        theme_scores[theme_name] = score
        theme_cards[theme_name] = matching_cards
    
    # Sort themes by potential
    sorted_themes = sorted(theme_scores.items(), key=lambda x: x[1], reverse=True)
    
    print(f"\n📈 CONTROL THEME POTENTIAL RANKING:")
    for i, (theme, score) in enumerate(sorted_themes):
        cards = theme_cards[theme]
        print(f"{i+1}. {theme}: {score} keyword matches, {len(cards)} cards")
        
        if cards:
            print(f"   📝 Top cards:")
            for card in cards[:4]:  # Show top 4 cards
                print(f"   • {card['name']} (CMC {card['cmc']}, {card['color']}) - {card['type']}")
                print(f"     Matches: {', '.join(card['matches'])}")
                print(f"     Text: {card['text']}")
                print()
    
    # Analyze current Azorius Control deck issues and alternatives
    print(f"\n⚖️  CURRENT AZORIUS CONTROL ISSUES:")
    print(f"   • Color identity problems (38.5% WU cards)")
    print(f"   • Off-color cards (Mutagenic Growth, Gitaxian Probe)")
    print(f"   • No board wipes for mass removal")
    print(f"   • Limited hard removal options")
    
    # Check specific improvements
    counterspell_potential = len(theme_cards.get('Counterspells', []))
    removal_potential = len(theme_cards.get('Removal/Bounce', []))
    board_wipe_potential = len(theme_cards.get('Board Wipes', []))
    card_draw_potential = len(theme_cards.get('Card Draw/Advantage', []))
    win_con_potential = len(theme_cards.get('Win Conditions', []))
    
    print(f"\n🔄 IMPROVEMENT OPPORTUNITIES:")
    if counterspell_potential >= 2:
        print(f"   ✅ COUNTERSPELLS: {counterspell_potential} additional counterspells available")
        print(f"      Could replace off-color cards with proper counters")
    
    if removal_potential >= 2:
        print(f"   ✅ REMOVAL: {removal_potential} additional removal/bounce spells available")
        print(f"      Better control tools than current options")
    
    if board_wipe_potential >= 1:
        print(f"   ✅ BOARD WIPES: {board_wipe_potential} board wipe(s) available")
        print(f"      Critical missing piece for control strategy")
    
    if win_con_potential >= 2:
        print(f"   ✅ WIN CONDITIONS: {win_con_potential} additional win conditions available")
        print(f"      Better finishers than current options")
    
    if card_draw_potential >= 3:
        print(f"   ✅ CARD ADVANTAGE: {card_draw_potential} additional card draw effects")
        print(f"      Already strong, but more options available")
    
    # Show problematic cards that could be replaced
    print(f"\n🔄 RECOMMENDED REPLACEMENTS:")
    current_problems = [
        "Mutagenic Growth (green Phyrexian mana - doesn't fit Azorius)",
        "Gitaxian Probe (colorless with Phyrexian - weak control tool)", 
        "Unbounded Potential (not a strong control spell)"
    ]
    
    for problem in current_problems:
        print(f"   ❌ Replace: {problem}")
    
    # Show best replacement options
    print(f"\n💡 BEST AVAILABLE UPGRADES:")
    all_good_cards = []
    
    # Prioritize counterspells, removal, and board wipes
    priority_themes = ['Counterspells', 'Board Wipes', 'Removal/Bounce']
    for theme in priority_themes:
        cards = theme_cards.get(theme, [])
        for card in cards[:2]:  # Top 2 from each priority theme
            all_good_cards.append(card)
    
    # Show top recommendations
    for i, card in enumerate(all_good_cards[:6]):  # Show top 6 recommendations
        print(f"   {i+1}. {card['name']} (CMC {card['cmc']}, {card['color']}) - {card['type']}")
        print(f"      {card['text']}")
        print()
    
    # Final recommendation
    print(f"\n🎯 OVERALL RECOMMENDATION:")
    total_improvements = counterspell_potential + removal_potential + board_wipe_potential
    if total_improvements >= 5:
        print(f"   🔄 SIGNIFICANT IMPROVEMENTS AVAILABLE")
        print(f"   📊 {total_improvements} better control cards could replace current weak options")
        print(f"   🎯 Focus on: True WU cards, counterspells, removal, board wipes")
        print(f"   📈 Potential score improvement: 62.5% → 80%+ with proper cards")
    else:
        print(f"   ⚠️  Limited improvements available in current card pool")

else:
    print("   ❌ Could not analyze unassigned cards - missing data")

🔍 UNASSIGNED AZORIUS CARDS ANALYSIS
📊 UNASSIGNED WHITE/BLUE CARDS: 17 total

🎯 CARD TYPE BREAKDOWN:
   • Unassigned creatures: 9
   • Unassigned instants: 3
   • Unassigned sorceries: 1

🎯 CONTROL THEME ANALYSIS OF UNASSIGNED CARDS:

📈 CONTROL THEME POTENTIAL RANKING:
1. Control Utility: 8 keyword matches, 4 cards
   📝 Top cards:
   • Prismatic Strands (CMC 3, W) - Instant
     Matches: tap, untap, prevent
     Text: prevent all damage that sources of the color of your choice would deal this turn...

   • Bubble Snare (CMC 1, U) - Enchantment - Aura
     Matches: tap, untap
     Text: kicker {2}{u} (you may pay an additional {2}{u} as you cast this spell.) | encha...

   • Weakstone's Subjugation (CMC 1, U) - Enchantment - Aura
     Matches: tap, untap
     Text: enchant artifact or creature | when this aura enters, you may pay {3}. if you do...

   • Witness Protection (CMC 1, U) - Enchantment - Aura
     Matches: protection
     Text: enchant creature | enchanted creature loses all a

In [8]:
# Analysis: Required Changes to consts.py and Scoring for Azorius Control
print("🔧 REQUIRED CHANGES ANALYSIS")
print("=" * 60)

print("📝 CURRENT AZORIUS CONTROL THEME DEFINITION:")
print("   'Azorius Control': {")
print("       'colors': ['W', 'U'],")
print("       'strategy': 'Classic control deck with counterspells, removal, and card draw',")
print("       'keywords': ['counter', 'destroy', 'exile', 'draw', 'instant', 'sorcery', 'board wipe'],")
print("       'archetype': 'Control'")
print("   }")

print(f"\n🔍 PROBLEMS WITH CURRENT DEFINITION:")
print("1. Keywords are too generic and match unintended cards")
print("2. Missing specific control terms that would find better cards")  
print("3. Card selection algorithm doesn't prioritize WU color identity")
print("4. No preference for true Azorius (WU) cards over mono-colored or off-color")

print(f"\n✅ RECOMMENDED CHANGES TO consts.py:")

print(f"\n1. IMPROVED KEYWORD LIST:")
improved_keywords = [
    # Counterspells - more specific
    'counter target spell', 'counter target', 'negate', 'cancel', 'dispel',
    # Removal - specific control removal
    'destroy target', 'exile target', 'return target', 'bounce', 'oblivion ring',
    # Card advantage
    'draw a card', 'draw two cards', 'scry', 'divination', 'fact or fiction',
    # Board control
    'tap target', 'detain', 'prevent damage', 'board wipe', 'wrath',
    # Azorius specific
    'white blue', 'azorius', 'flying', 'vigilance',
    # Control utility
    'flash', 'instant speed', 'end of turn', 'upkeep'
]

print("   Enhanced keywords for better card selection:")
for i, keyword in enumerate(improved_keywords[:10]):
    print(f"   • '{keyword}'")
if len(improved_keywords) > 10:
    print(f"   • ... and {len(improved_keywords) - 10} more specific control terms")

print(f"\n2. UPDATED THEME DEFINITION:")
print("   'Azorius Control': {")
print("       'colors': ['W', 'U'],")
print("       'strategy': 'Control deck with counterspells, removal, card draw, and efficient win conditions',")
print("       'keywords': [")
print("           # Core control spells")
print("           'counter target', 'negate', 'cancel', 'dispel',")
print("           'destroy target', 'exile target', 'oblivion ring', 'path to exile',")
print("           'return target', 'bounce', 'unsummon',")
print("           # Card advantage")  
print("           'draw a card', 'draw two', 'scry', 'divination',")
print("           # Board control")
print("           'tap target', 'detain', 'prevent damage', 'wrath', 'board wipe',")
print("           # Azorius identity")
print("           'white and blue', 'azorius', 'flying', 'vigilance', 'lifelink',")
print("           # Control timing")
print("           'flash', 'instant speed', 'end of turn', 'during upkeep'")
print("       ],")
print("       'archetype': 'Control',")
print("       'color_priority': 'strict'  # New field to prioritize WU cards")
print("   }")

print(f"\n3. SCORING ALGORITHM IMPROVEMENTS:")

print(f"\n   A. COLOR IDENTITY SCORING:")
print("   • Current: Counts any W or U cards as 'on-color'")
print("   • Problem: Phyrexian mana and colorless cards get counted")
print("   • Solution: Strict color checking")
print("   ")
print("   def calculate_color_score(deck, target_colors):")
print("       strict_matches = 0")
print("       for card in deck:")
print("           color = card.get('Color', '')")
print("           # Only count true WU, W, or U cards") 
print("           if any(c in color for c in target_colors) and not ('P' in color or color == ''):")
print("               strict_matches += 1")
print("       return strict_matches / len(deck)")

print(f"\n   B. KEYWORD MATCHING IMPROVEMENTS:")
print("   • Current: Simple string matching")
print("   • Problem: False positives (e.g., 'counter' matches '+1/+1 counter')")
print("   • Solution: Context-aware matching")
print("   ")
print("   def improved_keyword_matching(oracle_text, keywords):")
print("       score = 0")
print("       text = oracle_text.lower()")
print("       for keyword in keywords:")
print("           if keyword in text:")
print("               # Bonus for multiple keyword matches")
print("               score += 1")
print("               # Extra bonus for exact control phrases")
print("               if keyword in ['counter target spell', 'draw a card', 'destroy target']:")
print("                   score += 0.5")
print("       return score")

print(f"\n   C. CARD QUALITY SCORING:")
print("   • Add preference for:")
print("     - True multicolor WU cards (highest priority)")
print("     - Efficient control spells (low CMC counters/removal)")
print("     - Card advantage engines")
print("     - Win conditions with evasion (flying)")
print("   ")
print("   def calculate_card_quality(card, theme):")
print("       quality = 0")
print("       color = card.get('Color', '')")
print("       cmc = card.get('CMC', 0)")
print("       ")
print("       # Multicolor bonus")
print("       if 'WU' in color or 'UW' in color:")
print("           quality += 2  # Highest priority")
print("       elif 'W' in color or 'U' in color:")
print("           quality += 1")
print("       ")
print("       # Efficiency bonus for control spells")
print("       if cmc <= 3 and any(word in oracle_text for word in ['counter', 'destroy', 'exile']):")
print("           quality += 1")
print("       ")
print("       return quality")

print(f"\n4. CONSTRUCTION ALGORITHM CHANGES:")
print("   • Prioritize true WU cards in deck building")
print("   • Ensure minimum number of control spells (3-4)")
print("   • Limit creatures to 2-3 (control typical)")
print("   • Include at least 1 board wipe if available")
print("   • Prefer instant-speed interaction")

print(f"\n5. EXPECTED IMPROVEMENTS:")
print("   Current Azorius Control Score: 62.5%")
print("   With improved theme definition: ~75%")
print("   With better card selection: ~80-85%")
print("   ")
print("   Specific fixes:")
print("   • Color identity: 38% → 70%+ (remove Phyrexian cards)")
print("   • Control tools: 4 → 5-6 (better keyword matching)")
print("   • Card quality: Add Oblivion Ring, Lyev Skyknight, Silver Drake")
print("   • Remove: Mutagenic Growth, Gitaxian Probe, Unbounded Potential")

print(f"\n6. IMPLEMENTATION PRIORITY:")
print("   1. Update Azorius Control keywords in consts.py (HIGH)")
print("   2. Add color_priority field for strict WU matching (HIGH)")  
print("   3. Improve keyword matching in card selection (MEDIUM)")
print("   4. Add card quality scoring in construction (MEDIUM)")
print("   5. Update other control themes with similar improvements (LOW)")

print(f"\n🎯 SUMMARY:")
print("The main issue is the current keywords are too generic and the color")
print("matching isn't strict enough. By updating the theme definition with")
print("specific control terms and enforcing true WU color identity, the")
print("deck construction should automatically select better cards like")
print("Oblivion Ring and Lyev Skyknight instead of off-color Phyrexian cards.")

🔧 REQUIRED CHANGES ANALYSIS
📝 CURRENT AZORIUS CONTROL THEME DEFINITION:
   'Azorius Control': {
       'colors': ['W', 'U'],
       'strategy': 'Classic control deck with counterspells, removal, and card draw',
       'keywords': ['counter', 'destroy', 'exile', 'draw', 'instant', 'sorcery', 'board wipe'],
       'archetype': 'Control'
   }

🔍 PROBLEMS WITH CURRENT DEFINITION:
1. Keywords are too generic and match unintended cards
2. Missing specific control terms that would find better cards
3. Card selection algorithm doesn't prioritize WU color identity
4. No preference for true Azorius (WU) cards over mono-colored or off-color

✅ RECOMMENDED CHANGES TO consts.py:

1. IMPROVED KEYWORD LIST:
   Enhanced keywords for better card selection:
   • 'counter target spell'
   • 'counter target'
   • 'negate'
   • 'cancel'
   • 'dispel'
   • 'destroy target'
   • 'exile target'
   • 'return target'
   • 'bounce'
   • 'oblivion ring'
   • ... and 18 more specific control terms

2. UPDATED THEM